In [1]:
import os
import re
import tempfile
from monty.tempfile import ScratchDir

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_google_genai import GoogleGenerativeAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from llamp.atomate2.agents import MLFFMDAgent
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY", None)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4-turbo"

In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0.0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool(
    description=re.sub(
        r"\s+",
        " ",
        """A Python shell. Use this to execute python commands.
        Input should be a valid python command.
        
        TIPS:
        - If you want to see the output of a value, you should print it out
        with `print(...)`.
        - If you write any file to a local file system, you should ALWAYS 
        print the paths to the files as well.
        - If you plot any graph, you should show the graph, save it to local 
        and print the path to the image file.
        - Combine all your output string together and print once together in
        JSON format at the end of the code.
        """
    )
)]
tools += [
    MPThermoExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MLFFMDAgent(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
    r"\s+",
    " ",
    """You name is LLaMP and you are a helpful agent that can consult 
    materials-related data through Materials Project (MP) database, arXiv, 
    Wikipedia, and run calculations/simulations via a python REPL and MP atomate2 
    computational workflow.     
        
    If you get an error, debug your code and try again. Only use 
    the output of your code to answer the question. Ask user to clarify their 
    queries if needed. Please note that you don't have direct control over MP but 
    through multiple assistant agents to help you. You need to provide complete 
    context in the input for assistants to do their job. REFINE and CRITQUE the 
    output of the assistants if needed. You can end the conversation by saying "Bye".
    """,
    )
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

In [3]:



pattern = re.compile(r'\b' + re.escape("Bye") + r'\b[.!?]*')

query = """
Give me the formation energies of the following materials: Ag2O3, 
MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3. 
If there are multiple values, please give me the smallest value.
"""

# with ScratchDir(rootpath=os.getcwd()):
# with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
#     os.chdir(tmpdir)
#     while True:
query = query or input("Query:")
print(f"Query: {query}")
response = agent_executor.invoke(
    {
        "input": query,
    }
)
# if re.search(pattern, response["output"]):
#     break
# query = None

Query: 
Give me the formation energies of the following materials: Ag2O3, 
MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3. 
If there are multiple values, please give me the smallest value.



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPThermoExpert
Action Input: What is the formation energy of Ag2O3?Thought: Do I need to use a tool? Yes
Action: MPThermoExpert
Action Input: What is the formation energy of Ag2O3?

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_thermo__get",
  "action_input": {
    "formula": "Ag2O3"
  }
}
```Action:
```
{
  "action": "search_materials_thermo__get",
  "action_input": {
    "formula": "Ag2O3"
  }
}
```Error on search_materials_thermo__get: fields must be specified. Please revise arguments or try smaller request by specifying 'limit' in request.Action:
```
{
  "action": "search_materials_thermo__get",
  "action_input": {
    "formula": "Ag2O3",
    "fields": "material_id,for

Retrieving ThermoDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

[{'formula_pretty': 'Ag2O3', 'material_id': 'mp-546190', 'formation_energy_per_atom': -0.352642995166665, 'energy_above_hull': 0.019745315499998917}, {'formula_pretty': 'Ag2O3', 'material_id': 'mp-11872', 'formation_energy_per_atom': -0.18240765216666546, 'energy_above_hull': 0.1899806584999988}, {'formula_pretty': 'Ag2O3', 'material_id': 'mp-1046809', 'formation_energy_per_atom': -0.11580466916666551, 'energy_above_hull': 0.2565836414999989}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The formation energy per atom for Ag2O3 varies depending on the specific material. Here are the formation energies per atom for different materials with the formula Ag2O3: \n- Material ID mp-546190: -0.35264 eV/atom\n- Material ID mp-11872: -0.18241 eV/atom\n- Material ID mp-1046809: -0.11580 eV/atom\nPlease note that these values are for individual atoms within the compound and the total formation energy for the compound can be calculated by multiplying these values by the total number 

Retrieving ThermoDoc documents:   0%|          | 0/17 [00:00<?, ?it/s]

[{'formula_pretty': 'MgZn2', 'material_id': 'mp-1124', 'formation_energy_per_atom': -0.13063030499999861, 'energy_above_hull': 0.0}, {'formula_pretty': 'MgZn2', 'material_id': 'mp-978269', 'formation_energy_per_atom': -0.12303470999999828, 'energy_above_hull': 0.007595595000000399}, {'formula_pretty': 'MgZn2', 'material_id': 'mp-1094483', 'formation_energy_per_atom': -0.057302883333333256, 'energy_above_hull': 0.0733274216666655}, {'formula_pretty': 'MgZn2', 'material_id': 'mp-1094198', 'formation_energy_per_atom': -0.050285159999999905, 'energy_above_hull': 0.08034514499999879}, {'formula_pretty': 'MgZn2', 'material_id': 'mp-1094461', 'formation_energy_per_atom': -0.03852052999999985, 'energy_above_hull': 0.09210977499999884}, {'formula_pretty': 'MgZn2', 'material_id': 'mp-1094432', 'formation_energy_per_atom': -0.024907058333333214, 'energy_above_hull': 0.10572324666666555}, {'formula_pretty': 'MgZn2', 'material_id': 'mp-1094191', 'formation_energy_per_atom': -0.02111905666666658, 'e

...

[{'formation_energy_per_atom': -0.965501308541666, 'energy_above_hull': 0.0}, {'formation_energy_per_atom': -0.9638085802083344, 'energy_above_hull': 0.0016927283333316723}, {'formation_energy_per_atom': -0.9631120685416666, 'energy_above_hull': 0.002389239999999404}, {'formation_energy_per_atom': -0.9618197735416677, 'energy_above_hull': 0.003681534999998348}, {'formation_energy_per_atom': -0.961087964097222, 'energy_above_hull': 0.004413344444444078}, {'formation_energy_per_atom': -0.9582655135416663, 'energy_above_hull': 0.007235794999999712}, {'formation_energy_per_atom': -0.9510613952083323, 'energy_above_hull': 0.01443991333333372}, {'formation_energy_per_atom': -0.7991444898379617, 'energy_above_hull': 0.16635681870370433}, {'formation_energy_per_atom': -0.697958272708334, 'energy_above_hull': 0.2675430358333326}, {'formation_energy_per_atom': -0.6857841218749977, 'energy_above_hull': 0.27971718666666945}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The formation

Retrieving ThermoDoc documents:   0%|          | 0/11 [00:00<?, ?it/s]

[{'formation_energy_per_atom': -3.489696347166634, 'energy_above_hull': 0.0}, {'formation_energy_per_atom': -3.4896553571666344, 'energy_above_hull': 4.09900000004626e-05}, {'formation_energy_per_atom': -3.488399516833302, 'energy_above_hull': 0.0012968303333327214}, {'formation_energy_per_atom': -3.487739773166632, 'energy_above_hull': 0.001956574000002931}, {'formation_energy_per_atom': -3.487076905166634, 'energy_above_hull': 0.0026194420000020813}, {'formation_energy_per_atom': -3.482229675166633, 'energy_above_hull': 0.007466672000001395}, {'formation_energy_per_atom': -2.7754237491666656, 'energy_above_hull': 0.7142725979999689}, {'formation_energy_per_atom': -2.5106399891666653, 'energy_above_hull': 0.9790563579999692}, {'formation_energy_per_atom': -1.6830408051666665, 'energy_above_hull': 1.8066555419999677}, {'formation_energy_per_atom': -1.682236483166666, 'energy_above_hull': 1.8074598639999682}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The formation ener

Retrieving ThermoDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

[{'formula_pretty': 'CsPbI3', 'material_id': 'mp-540839', 'formation_energy_per_atom': -1.295464193931037, 'energy_above_hull': 0.0}, {'formula_pretty': 'CsPbI3', 'material_id': 'mp-1120768', 'formation_energy_per_atom': -1.2796408769310343, 'energy_above_hull': 0.015823316999999726}, {'formula_pretty': 'CsPbI3', 'material_id': 'mp-1069538', 'formation_energy_per_atom': -1.2704148499310342, 'energy_above_hull': 0.02504934400000014}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The formation energy per atom for CsPbI3 varies depending on the specific material entry. Here are the formation energies for different entries of CsPbI3 on Materials Project: -1.295464193931037 eV/atom for material_id mp-540839, -1.2796408769310343 eV/atom for material_id mp-1120768, and -1.2704148499310342 eV/atom for material_id mp-1069538."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The formation energy per atom for CsPbI3 varies depending on the specific material entry. 

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

Here are the smallest formation energies per atom for the requested materials:

- Ag2O3: -0.35264 eV/atom (Material ID mp-546190)
- MgZn2: -0.130630 eV/atom (Material ID mp-1124)
- GaAs: -0.446 eV/atom (Material ID mp-2534)
- GaN: -0.670049406875 eV/atom (Material ID mp-804)
- SiC: -0.20388216661842162 eV/atom (Material ID mp-1204356)
- BN: -1.4129789931250016 eV/atom (Material ID mp-604884)
- InSe: -0.6455260882812475 eV/atom (Material ID mp-21405)
- MoS2: -0.9655 eV/atom (Approximate value, specific Material ID not provided)
- BaTiO3: -3.4897 eV/atom (Approximate value, specific Material ID not provided)
- CsPbI3: -1.295464193931037 eV/atom (Material ID mp-540839)

Please note that the values for MoS2 and BaTiO3 are approximate and the specific Material IDs were not provided.


In [5]:

model = "gpt-3.5-turbo-1106"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-3.5-turbo-1106
The formation energies of the following materials are:

Ag2O3: -2.95 eV/atom
MgZn2: -0.21 eV/atom
GaAs: -1.81 eV/atom
GaN: -1.13 eV/atom
SiC: -4.46 eV/atom
BN: -5.41 eV/atom
InSe: -0.39 eV/atom
MoS2: -0.92 eV/atom
BaTiO3: -3.29 eV/atom
CsPbI3: -1.35 eV/atom


In [6]:

model = "gpt-4"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-4
I'm sorry but it seems there is a misunderstanding. Formation energy is a very specific property of materials usually measured in labs or simulated with computational models. It can vary based on different conditions (temperature, pressure, etc.) and it's not data that can be easily or directly looked up online. Therefore, I can't provide the formation energies for the materials you listed. You might need to consult a material science database or literature to find these values.


In [7]:

model = "gemini-pro"
llm = GoogleGenerativeAI(
    temperature=0.7,
    model=f"models/{model}",
    google_api_key=GOOGLE_API_KEY,
)


response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response)

gemini-pro
* Ag2O3: 0.24 eV/atom
* MgZn2: -0.04 eV/atom
* GaAs: -0.03 eV/atom
* GaN: -0.01 eV/atom
* SiC: -0.63 eV/atom
* BN: -0.28 eV/atom
* InSe: 0.04 eV/atom
* MoS2: -0.42 eV/atom
* BaTiO3: -0.29 eV/atom
* CsPbI3: -0.05 eV/atom


In [8]:
model = "llama3"
llm = ChatOllama(
    model=model,
    temperature=0.7,
    # callbacks=[StreamingStdOutCallbackHandler()],
    num_predict=-2,
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

llama3
Here are the formation energies for each of the materials you listed:

1. Ag2O3:
The formation energy is -1225 kJ/mol (source: Materials Project database)
2. MgZn2:
The formation energy is -144.6 kJ/mol (source: Materials Project database)

Note that there may be multiple values depending on the specific stoichiometry and phase of the material, but I've provided the smallest value available in the Materials Project database.

3. GaAs:
The formation energy is -1052.4 kJ/mol (source: Materials Project database)
4. GaN:
The formation energy is -1040.5 kJ/mol (source: Materials Project database)

Note that the formation energy of GaN can vary depending on the specific phase and stoichiometry, but this value is for the most stable phase.

5. SiC:
The formation energy is -1152.8 kJ/mol (source: Materials Project database)
6. BN:
The formation energy is -1200.4 kJ/mol (source: Materials Project database)

Note that there may be multiple values depending on the specific stoichiometry an